In [ ]:
import os
import GEOparse
import urllib.request
import gzip
import shutil
from pathlib import Path

# Sample accessions from GSE116222

RAW_DIR = Path("../data/raw")
RAW_DIR.mkdir(parents=True, exist_ok=True)


geo_id = "GSE116222"

gse = GEOparse.get_GEO(geo=geo_id, destdir=RAW_DIR, annotate_gpl=True)


print(list(gse.gsms.keys()))
print("Expected files: GSE116222_RAW.tar (contains all sample matrices)")

In [ ]:
import pandas as pd

records = []

for gsm_id, gsm in gse.gsms.items():
    meta = gsm.metadata
    record = {
        'sample_id': gsm_id,
        'title': meta.get('title', [''])[0],
        'condition': meta.get('characteristics_ch1', [''])[0],  # often contains condition info
        'patient_id': meta.get('characteristics_ch1', [''])[1] if len(meta.get('characteristics_ch1', [])) > 1 else ''
    }
    records.append(record)

metadata_df = pd.DataFrame(records)
print(metadata_df)
